In [1]:
import os
import sys

#pyspark-shell
#os.system("pip install kafka-python")

os.environ["SPARK_HOME"]="/home/cloudera/Downloads/spark-2.4.4-bin-hadoop2.6"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
#os.environ["HADOOP_HOME"] ="/usr/lib/hadoop/bin"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
#os.environ["PYSPARK_PYTHON"] = "/usr/local/anaconda/bin/python" 
#os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/local/anaconda/bin/python"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

#org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.3 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.3.4 pyspark-shell'



In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.window import Window
import pyspark.sql.functions as func
from pyspark.sql.types import TimestampType, StringType

spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

In [3]:
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()



In [4]:
# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

In [5]:
# Generate running word count
wordCounts = words.groupBy("word").count()

In [6]:
query = wordCounts \
    .writeStream \
    .queryName("agg1")\
    .outputMode("complete") \
    .format("memory") \
    .start()
spark.sql("select * from agg1").show() 


+----+-----+
|word|count|
+----+-----+
+----+-----+



In [7]:
from IPython.display import display, clear_output
from time import sleep
while True:
    clear_output(wait=True)
    display(query.status)
    display(spark.sql('SELECT * FROM agg1').show())
    sleep(1)

{'message': 'Waiting for data to arrive',
 'isDataAvailable': False,
 'isTriggerActive': False}

+-----+-----+
| word|count|
+-----+-----+
|hello|    1|
|  hai|    2|
+-----+-----+



None

KeyboardInterrupt: 